In [4]:
import torch
import gpytorch
import pandas as pd
import numpy as np
from scipy.io import savemat

subjins25_csv = pd.read_csv('SubjIns25.csv', header=None)
subjins25_data = subjins25_csv.values
subjins25_data = np.delete(subjins25_data, [0, 5, 10], axis=0)

torch.manual_seed(231127)
train_x = torch.tensor([22., 24., 25., 27., 30., 40., 50., 60., 70., 80., 90., 100.,
                        120., 140., 160., 180.], dtype=torch.float32)
subjins25 = torch.tensor(subjins25_data, dtype=torch.float32)
train_mean = torch.mean(torch.log(subjins25), dim=0)
train_std = torch.std(torch.log(subjins25), dim=0)

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RQKernel(
                alpha_prior=gpytorch.priors.SmoothedBoxPrior(0.8, 1.0),
                lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(0.01, 5.0)
            )
        )
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, None, likelihood)  # Set train_y to None initially

model.train()
likelihood.train()
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  
], lr=0.001)

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
training_iter = 10000
loss_threshold = 0.73
stop_training = False

for i in range(training_iter):
    optimizer.zero_grad()
    random_row_idx = torch.randint(0, subjins25.shape[0], (1,))
    train_y = (torch.log(subjins25[random_row_idx]) - train_mean) / train_std
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    if (i + 1) % 100 == 0:
        max_loss_idx = random_row_idx
        print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f  Max Loss Row: %d' % (
            i + 1, training_iter, loss.item(),
            model.covar_module.base_kernel.lengthscale.item(),
            model.likelihood.noise.item(),
            max_loss_idx.item()
        ))
    if loss.item() < loss_threshold:
        print('Loss is below the threshold. Stopping training.')
        stop_training = True
        break
    optimizer.step()
    if i + 1 >= training_iter:
        print('Exceeded maximum iterations. Stopping training.')
        stop_training = True
        break

if stop_training:
    model.eval()
    likelihood.eval()
    num_samples = 300
    total_samples = 5000000
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        test_x = train_x
        observed_pred = likelihood(model(test_x))

    new_test_x = train_x
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        new_samples = observed_pred.sample(torch.Size([total_samples]))
        pred_mean_rescaledback = torch.exp(observed_pred.mean * train_std + train_mean)
        lower_rescaledback = torch.exp(observed_pred.confidence_region()[0] * train_std + train_mean)
        upper_rescaledback = torch.exp(observed_pred.confidence_region()[1] * train_std + train_mean)

    new_samples_rescaled = torch.exp(new_samples * train_std + train_mean)
    new_samples_rescaled_double = new_samples_rescaled.double()
    xdatanp = np.array(train_x.unsqueeze(0))

    
# Save necessary variables to a MATLAB-compatible mat file
savemat('prepare_fig1_confidence_data.mat', {
    'test_x': test_x.numpy(),
    'lower_rescaledback': lower_rescaledback.numpy(),
    'upper_rescaledback': upper_rescaledback.numpy(),
    'subjins25': subjins25.numpy()
})


Iter 100/10000 - Loss: 1.200   lengthscale: 0.729   noise: 0.667  Max Loss Row: 14
Iter 200/10000 - Loss: 1.441   lengthscale: 0.766   noise: 0.636  Max Loss Row: 19
Iter 300/10000 - Loss: 1.517   lengthscale: 0.808   noise: 0.605  Max Loss Row: 1
Iter 400/10000 - Loss: 1.269   lengthscale: 0.860   noise: 0.585  Max Loss Row: 10
Iter 500/10000 - Loss: 1.957   lengthscale: 0.912   noise: 0.570  Max Loss Row: 8
Iter 600/10000 - Loss: 1.331   lengthscale: 0.964   noise: 0.555  Max Loss Row: 16
Iter 700/10000 - Loss: 1.291   lengthscale: 1.014   noise: 0.538  Max Loss Row: 3
Iter 800/10000 - Loss: 1.249   lengthscale: 1.060   noise: 0.519  Max Loss Row: 13
Iter 900/10000 - Loss: 1.171   lengthscale: 1.104   noise: 0.498  Max Loss Row: 7
Iter 1000/10000 - Loss: 1.162   lengthscale: 1.154   noise: 0.478  Max Loss Row: 15
Iter 1100/10000 - Loss: 1.150   lengthscale: 1.202   noise: 0.459  Max Loss Row: 15
Iter 1200/10000 - Loss: 1.975   lengthscale: 1.245   noise: 0.438  Max Loss Row: 18
Iter 